In [1]:
import json
import pickle as pkl
import numpy as np

In [6]:
path = r'/home/shensy/Code/python/doduo/data/turl_dataset/train.table_col_type.json'

In [3]:
with open (path, "r") as f:
    turl_type_test = json.load(f)
    print(turl_type_test[0])

['2728176-1', 'slovak parliamentary election, 1990', 2728176, 'results', '', ['party'], [[[[0, 0], [1314381, 'Public Against Violence']], [[1, 0], [482855, 'Christian Democratic Movement']], [[2, 0], [1381773, 'Slovak National Party']], [[3, 0], [11450391, 'Communist Party of Slovakia']], [[4, 0], [40341252, 'Coexistence – Hungarian Christian Democratic Movement']], [[5, 0], [1973976, 'Democratic Party']], [[6, 0], [47633978, 'Party of Greens']], [[9, 0], [2728242, 'Freedom Party']], [[14, 0], [765352, 'Czechoslovakian Socialist Party']]]], [['organization.organization']]]


In [2]:
path = r'/home/shensy/Code/python/doduo/data/ssy_test.coltype.pkl'

In [3]:
with open (path, "rb") as f:
    turl_type_test = pkl.load(f)
print(turl_type_test)

        table_id                                             labels  \
0      6675886-1                      [sports.sports_league_season]   
1      6675886-1                                    [people.person]   
2      6675886-1                        [organization.organization]   
3      6675886-1                      [business.business_operation]   
4      6675886-1                               [automotive.company]   
...          ...                                                ...   
1870  40723972-2  [location.administrative_division, location.lo...   
1871  40723972-2                        [organization.organization]   
1872  40723972-2  [government.political_party, organization.orga...   
1873  40723972-2                        [organization.organization]   
1874  40723972-2  [government.political_party, organization.orga...   

                                                   data  \
0     1998 1999 2000 2001 2002 2003 2004 2005 2006 2...   
1     Jackie Stewart Adrian F

In [64]:
try:
    with open ("mapping_dict.json", "r") as f:
        mdict = json.load(f)
except:
    mdict={}


with open (path, "rb") as f:
    turl_type_test = pkl.load(f)
    tlabels = turl_type_test["train"]["labels"].tolist()
    nlabels = [np.where(np.array(nl) == 1)[0].tolist() for nl in turl_type_test["train"]["label_ids"].tolist()]
    for i,t in enumerate(tlabels):
        flag = True
        while flag:
            flag=False
            for e in t:
                if e in mdict.keys():
                    t.remove(e)
                    nlabels[i].remove(mdict[e])
                    flag=True
    
    
    for i in range(len(tlabels)):
        if len(tlabels[i])==1:
            mdict[tlabels[i][0]]=nlabels[i][0]
        elif len(tlabels[i])>1:
            print(tlabels[i], nlabels[i])
    print(len(mdict))
    # res = [list(zip(temp[0],temp[1])) for temp in comb]
    
    # formatted_res = res[0]
    # for ele in res[1:]:
    #     formatted_res += ele
with open ("mapping_dict.json", "w") as f:
    json.dump(mdict,f)
    

255


In [4]:
from SPARQLWrapper import SPARQLWrapper, JSON
class DbSparql:
    def __init__(self,endpoint):
        self.endpoint=endpoint
        self.sparql = SPARQLWrapper(self.endpoint)
        self.sparql.setReturnFormat(JSON)
        self.sparql.setTimeout(10)

    def set_query(self,query):
        self.sparql.setQuery(query)
    def get_type(self,mention):
        if not mention:
            return []
        result_list = []  
        queryTypes = """
                SELECT DISTINCT str(?mtype) as ?mtype 
                WHERE {{
                    {{
                        ?val foaf:isPrimaryTopicOf <http://en.wikipedia.org/wiki/%s>.
                        ?val rdf:type ?mtype.
                    }}
                FILTER (strstarts(str(?mtype),'http://dbpedia.org/ontology/'))
                FILTER (strstarts(str(?val),'http://dbpedia.org/resource/'))
                }}
            """ % mention

        self.set_query(queryTypes)
        results = self.sparql.query().convert()

        for result in results["results"]["bindings"]:
            result_list.append(result["mtype"]["value"])

        if len(result_list) == 0:
            return []

        return result_list


class CaliSparql:

    def __init__(self,endpoint):
        self.endpoint=endpoint
        self.sparql = SPARQLWrapper(self.endpoint)
        self.sparql.setReturnFormat(JSON)
        self.sparql.setTimeout(10)

    def set_query(self,query):
        self.sparql.setQuery(query)


    def get_type(self,mention):
        if not mention:
            return []
        result_list = []
        queryTypes = """
                SELECT DISTINCT str(?mtype) as ?mtype 
                WHERE {{
                    {{
                        <http://caligraph.org/resource/%s> rdf:type ?mtype.
                    }}
                FILTER (strstarts(str(?mtype),'http://caligraph.org/ontology/'))
                }}
            """ % mention

        self.set_query(queryTypes)
        results = self.sparql.query().convert()


        for result in results["results"]["bindings"]:
            result_list.append(result["mtype"]["value"])

        if len(result_list) == 0:
            return []

        return result_list


In [17]:
import requests, json
import pandas as pd
import numpy as np

def wiki_id2url(pgid:int):   
    pgid= str(pgid)
    x = requests.get('https://en.wikipedia.org/w/api.php?action=query&prop=info&pageids=%s&inprop=url&format=json' % pgid)
    return x.json()["query"]["pages"][pgid]["fullurl"]

def wiki_url2type(wikiurl:str):
    dbq=DbSparql(endpoint='https://dbpedia.org/sparql')
    caliq=CaliSparql(endpoint='http://caligraph.org/sparql')
    return dbq.get_type(wikiurl.split("/")[-1])+caliq.get_type(wikiurl.split("/")[-1])

turl_dataset = json.load(open('/home/shensy/Code/python/doduo/data/turl_dataset/test.table_col_type.json',"r"))
idx_dict = json.load(open('./mapping_dict.json',"r"))

try:
    wiki_id2u = json.load(open('/home/shensy/Code/python/doduo/data/wiki_id2url.json',"r"))
except:
    wiki_id2u = {}
try:
    wiki2heter = json.load(open('/home/shensy/Code/python/doduo/data/wikiurl2type.json',"r"))
except:
    wiki2heter = {}

fineg_pre_list = []
for table in turl_dataset[:3]:
    [table_idx, page_title, table_id, headline, table_name, header, data, gd_header] = table
    
    table_id  = [table_idx for i in range(len(data))]
    list_col_data = [[] for i in range(len(data))]
    list_type_dict = [{} for i in range(len(data))]
    labels = gd_header
    header = header
    
    for i,row in enumerate(data):
        for cell in row:
            list_col_data[i].append(cell[-1][-1]) 
            if cell[-1][0] in wiki_id2u.keys():
                key = wiki_id2u[cell[-1][0]]
            else:
                try:
                    key = wiki_id2url(cell[-1][0])
                except:
                    continue
                wiki_id2u[cell[-1][0]] = key
            if key in wiki2heter.keys():
                type_set = wiki2heter[key]
            else:
                try:
                    type_set = wiki_url2type(key)
                except:
                    continue
                wiki2heter[key] = type_set
            for t in type_set:
                try:
                    list_type_dict[i][t]+=1
                except:
                    list_type_dict[i][t]=1
    
    list_col_data = [' '.join(l) for l in list_col_data]
    label_ids = [np.zeros(len(idx_dict)) for i in range(len(data))]
    for i, ids in enumerate(label_ids):
        ids[[idx_dict[t] for t in gd_header[i]]] = 1
    fineg_pre_list += zip(table_id,labels,list_col_data,label_ids,header,list_type_dict)

ssy_df = pd.DataFrame(fineg_pre_list,
                    columns=[
                        "table_id", "labels", "data",
                        "label_ids", "header", "entities"
                    ])

with open('/home/shensy/Code/python/doduo/data/wiki_id2url.json',"w") as f:
    json.dump(wiki_id2u,f)
with open('/home/shensy/Code/python/doduo/data/wikiurl2type.json',"w") as f:
    json.dump(wiki2heter,f)

In [2]:
text = "https://zh.wikipedia.org/wiki/%E6%AF%9B%E6%B3%BD%E4%B8%9C"

print(text.split("/")[-1])

%E6%AF%9B%E6%B3%BD%E4%B8%9C


In [12]:
import csv, json
import pandas as pd
from turtle import color
# csv_iterator=csv.reader(open("./sato_cv_4.csv",'r',encoding="utf8"))

# type2num=dict()
# for line in csv_iterator:
#     try:
#         type2num[line[3]]=line[2]
#     except:
#         continue
# print(type2num)


x_data=[]
y_data=[]
x_num=[]
res_iterator=json.load(open("../eval/turlturl-recl_mosato_bert_bert-base-uncased-bs16-ml-32__turl-1.00_turl-re-1.00=turl.json",'r',encoding="utf8"))

print(res_iterator["f1_macro"]["ts_class_f1"][16])

# for i in range(len(res_iterator["f1_macro"]["ts_class_f1"])):
#     if res_iterator["f1_macro"]["ts_class_f1"][i]<0.7:
#         y_data.append(res_iterator["f1_macro"]["ts_class_f1"][i])
#         x_data.append(type2num[str(i)])
#         x_num.append(i)
#     print(i, res_iterator["f1_macro"]["ts_class_f1"][i])

# res=[]
# for i in x_num:
#     print(sum(res_iterator["f1_macro"]["confusion_matrix"][i]),type2num[str(i)])
#     res.append(res_iterator["f1_macro"]["confusion_matrix"][i])
# dat = pd.DataFrame(data=res) 

# dat.to_csv("bad_acc_sato.csv")

0.8571428571428571


In [13]:
k = np.array([[[1, 0],
        [0, 1]],

       [[1, 0],
        [0, 1]],

       [[0, 1],
        [1, 0]]])
print(k.sum(axis=0))
print(k.shape)

[[2 1]
 [1 2]]


In [19]:
import re

a = re.compile(r"(A|[1-9])")

for i in a.finditer("....123...A"):
    print(i.group(), i.start())

1 4
2 5
3 6
A 10
